<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250"/>

# Cours TAL - Laboratoire 3
# Analyse syntaxique du français

**Objectifs**
1.	Appliquer un analyseur syntaxique **de dépendances** sur des données de test en français et observer les scores.  Entraîner cet analyseur sur des données d'entraînement adaptées, et déterminer si les scores se sont améliorés ou non.
2.	Importer des fichiers annotés dans NLTK, les transformer en **graphes de dépendances**, et trouver les paires sujet-verbe les plus fréquentes.
3.	En appliquant un analyseur **en constituants**, extraire tous les groupes nominaux d'un corpus en français.

**Instructions initiales**

Vous utiliserez les mêmes données que pour le labo 2, car elles contiennent aussi des informations syntaxiques.  Pour rappel, ces données sont disponibles dans [l'archive ZIP fournie sur Cyberlearn au labo 2](https://cyberlearn.hes-so.ch/pluginfile.php/3493142/mod_assign/introattachment/0/UD_French-GSD-withBlankLines.zip?forcedownload=1).  Ces textes en français proviennent du projet [Universal Dependencies (UD)](https://github.com/UniversalDependencies/UD_French-GSD).  Le fichier `fr-ud-train.conllu3` est destiné à l'entraînement, `fr-ud-dev.conllu3` aux tests/réglages préliminaires, et `fr-ud-test.conllu3` à l'évaluation finale.

## 1. Appliquer un analyseur de dépendances.  Entraîner l'analyseur sur les données UD

Dans cette 1ère partie, vous allez appliquer (puis entraîner) le [Stanford Dependency Parser](https://nlp.stanford.edu/software/nndep.html) qui utilise un réseau de neurones.  Comme au Labo 2, vous aurez besoin du code, d'un modèle entraîné, et de la documentation.

* **code Java** : on peut le télécharger dans un [package fourni par Stanford](https://nlp.stanford.edu/software/lex-parser.html) avec d'autres parsers et modèles, mais *on vous recommande* de simplement télécharger l'archive `stanford-corenlp-3.9.2.jar` depuis le [site Maven de Stanford CoreNLP](https://search.maven.org/artifact/edu.stanford.nlp/stanford-corenlp/3.9.2/jar) (choisir *Browse* ou *Downloads*)
* **modèles** : plusieurs modèles sont disponibles sur le site Maven, donc un package pour le français de 272 MB ; comme vous n'aurez besoin que du modèle (UD) pour Universal Dependencies, *on vous recommande* de télécharger le modèle pour UD pré-entraîné `UD_French.gz` mis à disposition pour le cours TAL sur [drive.switch.ch](https://drive.switch.ch/index.php/s/OqlSRUCSBvqKg3O)
* dans la [documentation du parser](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/parser/nndep/DependencyParser.html) regarder surtout le `main()` et les exemples à la fin

**Travail demandé**

Utiliser la documentation pour effectuer les tâches suivantes.

1. Exécuter le parser en Java (avec une commande externe `!java -cp ...` comme au labo 2) en l'appliquant au fichier UD de *test* en français.  Quels sont les deux scores obtenus et que signifient-ils ?
2. Entraîner le parser pour générer un nouveau modèle à partir des données d’entraînement.
3. Exécuter le parser que vous avez entraîné sur le même fichier de test qu'en (1).  Quels scores obtient-il et comment cela se compare avec le modèle fourni ?

**Note** : il est aussi possible d'appeler ce parser depuis NLTK, à condition de lancer le [serveur CoreNLP](https://stanfordnlp.github.io/CoreNLP/corenlp-server.html) et d'utiliser une instance de la classe `CoreNLPDependencyParser` ainsi : `dep_parse = next(dep_parser.raw_parse("La maison est petite."))`.

In [1]:
# Veuillez écrire ici la commande pour tester le parser avec le modèle pré-entraîné.
! java -cp stanford-corenlp-3.9.2.jar edu.stanford.nlp.parser.nndep.DependencyParser -model UD_French.gz  -testFile fr-ud-dev.conllu3 -outFile dependenciesOutputFile.txt 
#UAS = 57.2195(Unlabeled Attachment Scores): fait référence au pourcentage de mots qui ont des index sont corrects 
#LAS = 43.6722 (Labeled Attachment Scores):  fait référence au pourcentage de mots dont les index et les étiquettes de dépendance sont tous deux corrects 

Loading depparse model: UD_French.gz ... 
###################
#Transitions: 81
#Labels: 40
ROOTLABEL: root
PreComputed 99996, Elapsed Time: 12.584 (s)
Initializing dependency parser ... done [13.7 sec].
Test File: fr-ud-dev.conllu3
OOV Words: 2716 / 35771 = 7.59%
UAS = 57.2195
LAS = 43.6722
DependencyParser parsed 35771 words in 1478 sentences in 16.5s at 2170.7 w/s, 89.7 sent/s.


**Indications pour l'entraînement**
* pensez donner un nouveau nom au modèle qui sera créé (attention à l'écrasement)
* notez que vous utiliserez à la fois `train` *et* `dev` (voir la documentation)
* évitez un output trop volumineux dans le notebook : redirigez le tout dans un fichier `output.txt` en faisant suivre la commande par `>output.txt 2>&1` (valable sous Linux et Windows)
* plusieurs options indiquées dans la documentation peuvent être utiles
  * `-wordCutOff 3` pour traiter seulement les mots apparaissant plus de 3 fois, ce qui évite notamment le problème des nombres écrits avec un espace (apparaissant 1 fois)
  * `-trainingThreads 4` pour utiliser pleinement votre processeur : indiquez le maximum selon votre modèle
  * `-maxIter 5000` pour arrêter l'entraînement après 5000 itérations (on peut commencer avec beaucoup moins pour estimer le temps nécessaire)

In [54]:
# Veuillez écrire ici la commande pour entraîner l'analyseur sur le fichier 'train' et créer un nouveau modèle.
# Pour ne pas bloquer le notebook, vous pouvez l'exécuter en ligne de commande.
! java -cp stanford-corenlp-3.9.2.jar edu.stanford.nlp.parser.nndep.DependencyParser -trainFile fr-ud-train.conllu3 -devFile fr-ud-dev.conllu3  -model my_UD_French2.gz -wordCutOff 3 -trainingThreads 4 -maxIter 5000  > output.txt 2>&1


Le processus ne peut pas acc‚der au fichier car ce fichier est utilis‚ par un autre processus.


In [5]:
# Veuillez écrire ici la commande pour tester l'analyseur avec le nouveau modèle, puis commentez les résultats ci-dessous.
#! java -cp stanford-corenlp-3.9.2.jar edu.stanford.nlp.parser.nndep.DependencyParser -model my_UD_French2.gz  -testFile fr-ud-dev.conllu3 -outFile newdependenciesOutputFile.txt 

Loading depparse model: my_UD_French2.gz ... 
Loaded TreebankLanguagePack: edu.stanford.nlp.trees.PennTreebankLanguagePack
###################
#Transitions: 91
#Labels: 45
ROOTLABEL: root
PreComputed 100000, Elapsed Time: 1.201 (s)
Initializing dependency parser ... done [2.2 sec].
Test File: fr-ud-dev.conllu3
OOV Words: 4724 / 35771 = 13.21%
UAS = 80.6883
LAS = 74.6079
DependencyParser parsed 35771 words in 1478 sentences in 3.0s at 12113.4 w/s, 500.5 sent/s.


**Question subsidiaire**

Écrivez des instructions pour extraire du fichier de logs du *DependencyParser*, une fois l'entraînement fini, l'évolution des scores suivants :
* le score UAS sur l'ensemble de développement (écrit par défaut tous les 100 itérations)
* le taux de correction et la fonction de coût (sur l'ensemble d'entraînement) aux mêmes itérations que les scores UAS

Affichez ces trois valeurs sur un graphe, en fonction du nombre d'itérations sur l'axe horizontal.

In [45]:
# Veuillez écrire ici le code qui extrait les valeurs.
import os
import nltk
chaines = ["UAS",
           "Cost"] # Texte à rechercher

uas, cost, correct = [], [], []
file_UD_French2 = "output.txt"
#files = [fr_ud_dev, fr_ud_train, fr_ud_test]

fd = open(file_UD_French2, "r",encoding='utf-8')
for line in fd.readlines():
    for chaine in chaines:
        if chaine in line:
            tokens = nltk.word_tokenize(line)
            print(tokens)
fd.close()

['Cost', '=', '3.776243120931692', ',', 'Correct', '(', '%', ')', '=', '0.17140000000000077']
['UAS', ':', '30.421290989908027']
['Cost', '=', '3.8416074888454697', ',', 'Correct', '(', '%', ')', '=', '0.15800000000000042']
['Cost', '=', '3.8474937065489137', ',', 'Correct', '(', '%', ')', '=', '0.15490000000000032']
['Cost', '=', '3.8281489435230087', ',', 'Correct', '(', '%', ')', '=', '0.1610000000000005']
['Cost', '=', '3.822269356707752', ',', 'Correct', '(', '%', ')', '=', '0.16200000000000053']
['Cost', '=', '3.8312678711001364', ',', 'Correct', '(', '%', ')', '=', '0.1609000000000005']
['Cost', '=', '3.852502059751854', ',', 'Correct', '(', '%', ')', '=', '0.15790000000000043']
['Cost', '=', '3.8244982666043206', ',', 'Correct', '(', '%', ')', '=', '0.16210000000000052']
['Cost', '=', '3.8623762263144226', ',', 'Correct', '(', '%', ')', '=', '0.15530000000000033']
['Cost', '=', '3.815487596856908', ',', 'Correct', '(', '%', ')', '=', '0.16560000000000064']
['Cost', '=', '3.8547

In [ ]:
import matplotlib.pyplot
%matplotlib inline

In [ ]:
# Écrivez ici le code affichant les trois courbes.  
# Pensez-vous qu'il soit utile d'entraîner le parser plus longtemps ?


## 2.	Extraire des données UD les paires sujet-verbe les plus fréquentes

Dans cette 2e partie, vous allez d'abord importer les données UD avec l'annotation syntaxique *en dépendances*, grâce à un convertisseur appelé `nltk.parse.DependencyGraph`.  Celui-ci permet de transformer une phrase annotée au format CONLL (c'est-à-dire sur plusieurs lignes avec un mot et ses annotations par ligne) en un graphe de dépendances.  Vous pourrez ensuite faire des statistiques de fréquences sur les trois fichiers `fr-ud-XXX.conllu3`.

**Note 1** : on ne peut pas utiliser `ConllCorpusReader` comme dans le labo 2, parce que l'information syntaxique des données UD françaises est faite de *dépendances*, or ConllCorpusReader s'attend à trouver une information de faite de *constituants*.

**Note 2** : on utilise ici l'annotation fournie avec le corpus, mais on aurait pu aussi effectuer l'analyse syntaxique avec le parser de Stanford obtenu ci-dessus, pour un texte nouveau.

**Travail demandé**
* Lisez d'abord les [explications fournies par NLTK](http://www.nltk.org/howto/dependency.html) (début de la page)  pour transformer une phrase au format CONLL en un graphe de dépendances (classe `DependencyGraph`) et accéder aux informations de ce graphe.  Ensuite :
* Parcourez le(s) fichier(s) UD phrase par phrase (attention aux espaces dans certains nombres), et créez un graphe de dépendance pour chaque phrase.  Attention, il faut préciser que la relation de plus haut niveau est 'root' (et non 'ROOT').
* Sélectionnez les triplets ayant une relation *nsubj* (entre sujet et verbe).
* Donnez les 10 triplets les plus fréquents dans tout le corpus (train + dev + test).

In [86]:
# Veuillez écrire ici le code pour ouvrir le fichier 'fr-ud-test.conllu3'
# et lire son contenu dans un tableau de strings (un string = les lignes de chaque phrase).
# Indiquez le nombre de phrases et affichez la phrase d'indice 400 comme exemple.
import os
import nltk
tab_UD_test_conllu3 = []
file_UD_test_conllu3 = 'fr-ud-test.conllu3'
#files = [fr_ud_dev, fr_ud_train, fr_ud_test]
with open(file_UD_test_conllu3, "r" ,encoding='utf-8') as f_read:
    for line in f_read:
        line = line.strip()  #suppression du retour charriot en fin de ligne
        if line:
            tab_UD_test_conllu3.append(line)
print('Nombre Ligne: ',len(tab_UD_test_conllu3))
print('phrase indice 400: ', tab_UD_test_conllu3[400])



Nombre Ligne:  10298
phrase indice 400:  9	aussi	aussi	ADV	_	_	10	advmod	_	_


In [89]:
# Dans les chaînes obtenues, il reste des mots avec des espaces, que l'on veut supprimer.  
# Veuillez écrire ici la ligne de code pour supprimer tous les espaces.
tab_UD_test_final = []
for element in tab_UD_test_conllu3:
    tab_UD_test_final.append(element.replace(" ", ""))
print('phrase indice 400: ', tab_UD_test_final[400])

phrase indice 400:  9	aussi	aussi	ADV	_	_	10	advmod	_	_


In [90]:
from nltk.parse import DependencyGraph

In [117]:
# Veuillez écrire ici le code pour créer un tableau de graphes de dépendances (1 par phrase)
# et cherchez les relations de type 'nsubj' dans la phrase d'indice 400 du fichier 'test'.

tab_DependencyGrah = []
for element in tab_UD_test_final:
    tab_DependencyGrah.append(DependencyGraph(element, top_relation_label='root'))
print(tab_DependencyGrah[400])

tab_DependencyGrah[400].tree().pprint()

#for head, rel, dep in tab_DependencyGrah[400].triples():
    # print(
       #  '({h[0]}, {h[1]}), {r}, ({d[0]}, {d[1]})'
        # .format(h=head, r=rel, d=dep)
    # )

defaultdict(<function DependencyGraph.__init__.<locals>.<lambda> at 0x0000014B010615E0>,
            {0: {'address': 0,
                 'ctag': 'TOP',
                 'deps': defaultdict(<class 'list'>, {'root': []}),
                 'feats': None,
                 'head': None,
                 'lemma': None,
                 'rel': None,
                 'tag': 'TOP',
                 'word': None},
             9: {'address': 9,
                 'ctag': 'ADV',
                 'deps': defaultdict(<class 'list'>, {}),
                 'feats': '_',
                 'head': 10,
                 'lemma': 'aussi',
                 'rel': 'advmod',
                 'tag': '_',
                 'word': 'aussi'},
             10: {'address': None,
                  'ctag': None,
                  'deps': defaultdict(<class 'list'>, {'advmod': [9]}),
                  'feats': None,
                  'head': None,
                  'lemma': None,
                  'rel': None,
          

TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Veuillez écrire ici le code pour extraire la paire (sujet, verbe)
# de la première relation 'nsubj' trouvée ci-dessus.  Il servira ci-dessous.


In [ ]:
from nltk.probability import FreqDist

In [ ]:
# En utilisant les FreqDist, veuillez afficher les 10 paires 
# (nom, verbe) les plus fréquentes dans le fichier 'test'.
# Pensez convertir les mots en minuscule pour consolider le comptage.


In [ ]:
# Veuillez consolider ces résultats en regroupant les trois fichiers 'train', 'dev' et 'test'.
# Affichez cette fois-ci les 20 paires (sujet, verbe) les plus fréquentes.


## 3.	Appliquer un analyseur syntaxique en constituants. Extraire les groupe nominaux

Dans cette 3e partie, vous utiliserez l'analyseur syntaxique en constituants appelé **LexicalizedParser**, toujours fourni parmi les outils CoreNLP de Stanford, et [documenté ici](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/parser/lexparser/LexicalizedParser.html).  

Le code du parser figure dans le même fichier `.jar` que ci-dessus, et vous pouvez télécharger le modèle `frenchFactored.ser.gz` depuis le [Switch Drive](https://drive.switch.ch/index.php/s/R6gXPifDupAbo3l) de l'enseignant (4 MB).  

L'objectif de cette 3e partie est d'extraire les **groupes nominaux** les plus fréquents d'un texte.


**Note** : Pour travailler avec les outils CoreNLP depuis NLTK, la solution la plus récente consiste à démarrer un [serveur CoreNLP](https://stanfordnlp.github.io/CoreNLP/corenlp-server.html), soit depuis NLTK soit en ligne de commande.  Pour mémoire, les commandes sont `java -mx4g -cp "stanford-corenlp-3.9.2.jar;stanford-french-corenlp-2018-01-31-models.jar" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -serverProperties StanfordCoreNLP-french.properties -port 9000 -timeout 15000` ou alors `CoreNLPServer("stanford-corenlp-3.9.2.jar", "stanford-french-corenlp-2018-01-31-models.jar").start()`. Les [modèles pour le français](http://central.maven.org/maven2/edu/stanford/nlp/stanford-corenlp/3.9.2/) font 277 MB.  -- Mais nous n'utiliserons pas cela dans ce qui suit.

In [ ]:
# Veuillez écrire la ligne de commande (java) qui effecte l'analyse syntaxique
# en constituants d'un court texte de votre choix (10 phrases).  Choisissez
# 'oneline' comme format des résultats et écrivez-les dans un ficher.


Vous allez utiliser la classe BracketParseCorpusReader de NLTK pour importer les lignes contenant les analyses syntaxiques vers des objets `Tree` sous NLTK, auquels vous pourrez appliquer diverses fonctions (voir https://www.nltk.org/_modules/nltk/tree.html#Tree), comme demandé ci-dessous.

In [ ]:
from nltk.corpus.reader.bracket_parse import BracketParseCorpusReader

In [ ]:
# Écrivez le code qui importe le fichier de résultats du LexicalizedParser comme une liste d'arbres.
# Cherchez dans la documentation de Tree une fonction d'affichage joli et affichez le premier arbre.


In [ ]:
# Écrivez le code qui extrait les groupes nominaux de toutes les phrases (représentés
# dans votre liste comme des strings), et qui affiche les 10 les plus fréquents.


Veuillez enfin répéter l'expérience avec un texte plus long (une page Wikipedia ou un texte du projet Gutenberg) : indiquez les 20 GN les plus fréquents, et le temps approximatif nécessaire pour le parsing.

## Fin du laboratoire 3

Merci de nettoyer votre feuille, exécuter une dernière fois toutes les instructions, sauvegarder le résultat, et le soumettre sur Cyberlearn.